In [3]:
import pandas as pd

raw_train_data = pd.read_csv('ratings_train.txt', header=0, delimiter='\t', quoting=3).dropna(subset=['document'])
raw_test_data = pd.read_csv('ratings_test.txt', header=0, delimiter='\t', quoting=3).dropna(subset=['document'])

raw_train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
print(raw_train_data.shape, raw_test_data.shape)

(149995, 3) (49997, 3)


In [ ]:
def clean_documents(df):
    size = len(df)
    for i in range(size):
        clean_documents